In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import random
import threading

In [2]:
# Setup global configurations
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)

In [3]:
# Load empty board
df = pd.DataFrame(np.array(list(" " * 15 * 35)).reshape(15,35))

In [4]:
# Initialize board
output = widgets.Output()
board = widgets.HBox([output], layout=Layout(height='450px'))
def refreshBoard(source=None):
    output.clear_output(wait=True)
    with output:
        display(df)

In [5]:
# Start game 
def startGame():
    display(board)
    refreshBoard()

In [6]:
startGame()